# Garmin CSV to Day One CSV

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tkinter as tk
import datetime
from tkinter import filedialog
import ipywidgets as widgets

### Import Garmin CSV

In [ ]:
root = tk.Tk()
root.withdraw()
root.call("wm", "attributes", ".", "-topmost", True)
file_path = filedialog.askopenfilename(
    title="Select Garmin CSV", initialdir=os.getcwd()
)
print("Selected file:")
print(file_path)

### Import dataframe

In [ ]:
df_gar = pd.read_csv(file_path, sep=",", header=0, index_col=None)

### Summary statistics

In [ ]:
n_workouts = len(df_gar["Activity Type"])
print("Total number of workouts: {}".format(n_workouts))

activity_types = np.unique(df_gar["Activity Type"])
n_activity_types = np.zeros(len(activity_types))
for i, activity_type in enumerate(activity_types):
    n_activity_types[i] = len(
        df_gar["Activity Type"][df_gar["Activity Type"] == activity_type]
    )

activity_type_sort_idx = np.flip(np.argsort(n_activity_types))
activity_types_sorted = activity_types[activity_type_sort_idx]
n_activity_types_sorted = n_activity_types[activity_type_sort_idx]

for i, activity_type in enumerate(activity_types_sorted):
    print(
        "     {}: {} {}".format(
            activity_type,
            int(n_activity_types_sorted[i]),
            ("activities" if int(n_activity_types_sorted[i]) > 1 else "activity"),
        )
    )

print(" ")

### Select start date

In [ ]:
dates_unique = np.unique(df_gar["Date"])
dates_unique_str = np.array([date[:10] for date in dates_unique])

date_select = widgets.Select(
    options=dates_unique_str,
    value=dates_unique_str[0],
    description="Start date:",
    style={"description_width": "initial"},
)

confirm_button = widgets.Button(description="Confirm", disabled=False)
display(date_select, confirm_button)


def confirm_button_func(a):
    global start_date_str
    start_date_str = date_select.value

    print("Selected start date: " + start_date_str)
    return start_date_str


confirm_button.on_click(confirm_button_func)

### Rearrange dataframe

In [ ]:
start_date = df_gar["Date"][np.flip(dates_unique_str == start_date_str)].iloc[0]
start_date_idx = np.where(df_gar["Date"] == start_date)[0][0]
df_gar_select = df_gar.iloc[: start_date_idx + 1]

df_gar_select["Date"] = [
    (datetime.datetime.strptime(item, "%Y-%m-%d %H:%M:%S")).isoformat() + ".000+02:00"
    for item in df_gar_select["Date"]
]

In [ ]:
date_DO = np.array(df_gar_select["Date"])
text_DO = []

count = 0
for i, date in enumerate(date_DO):

    if df_gar_select["Activity Type"][df_gar_select["Date"] == date].values in [
        "Running",
        "Hiking",
        "Walking",
    ]:
        row = df_gar_select[df_gar_select["Date"] == date]

        activity_str = "{}: {} \n{} km in {}, average pace: {} min/km, max. pace: {} min/km\nAverage HR: {} bpm, max. HR: {} bpm \nTotal ascent: {} m".format(
            row["Activity Type"].values[0],
            row["Title"].values[0],
            row["Distance"].values[0],
            row["Time"].values[0],
            row["Avg Speed"].values[0],
            row["Max Speed"].values[0],
            (
                int(row["Avg HR"].values[0])
                if not np.isnan(row["Avg HR"].values[0])
                else "-"
            ),
            (
                int(row["Max HR"].values[0])
                if not np.isnan(row["Max HR"].values[0])
                else "-"
            ),
            row["Total Ascent"].values[0],
        )

        text_DO.append(activity_str)
    elif df_gar_select["Activity Type"][df_gar_select["Date"] == date].values in [
        "Cycling",
        "Mountain Biking",
    ]:
        row = df_gar_select[df_gar_select["Date"] == date]

        activity_str = "{}: {} \n{} km in {}, average speed: {} km/h, max. speed: {} km/h\nAverage HR: {} bpm, max. HR: {} bpm \nTotal ascent: {} m".format(
            row["Activity Type"].values[0],
            row["Title"].values[0],
            row["Distance"].values[0],
            row["Time"].values[0],
            row["Avg Speed"].values[0],
            row["Max Speed"].values[0],
            (
                int(row["Avg HR"].values[0])
                if not np.isnan(row["Avg HR"].values[0])
                else "-"
            ),
            (
                int(row["Max HR"].values[0])
                if not np.isnan(row["Max HR"].values[0])
                else "-"
            ),
            row["Total Ascent"].values[0],
        )

        text_DO.append(activity_str)
    elif df_gar_select["Activity Type"][df_gar_select["Date"] == date].values in [
        "Open Water Swimming",
        "Pool Swim",
    ]:
        row = df_gar_select[df_gar_select["Date"] == date]

        activity_str = "{}: {} \n{} m in {}, average speed: {} min/100m, max. speed: {} min/100m".format(
            row["Activity Type"].values[0],
            row["Title"].values[0],
            row["Distance"].values[0],
            row["Time"].values[0],
            row["Avg Speed"].values[0],
            row["Max Speed"].values[0],
        )

        text_DO.append(activity_str)
    else:
        row = df_gar_select[df_gar_select["Date"] == date]

        activity_str = "{}: {} \nDuration: {}\nAverage HR: {} bpm, max. HR: {} bpm ".format(
            row["Activity Type"].values[0],
            row["Title"].values[0],
            row["Time"].values[0],
            (
                int(row["Avg HR"].values[0])
                if not np.isnan(row["Avg HR"].values[0])
                else "-"
            ),
            (
                int(row["Max HR"].values[0])
                if not np.isnan(row["Max HR"].values[0])
                else "-"
            ),
        )

        text_DO.append(activity_str)

df_DO = pd.DataFrame({"date": date_DO, "text": text_DO})

### Export to CSV

In [ ]:
csv_name = "GARtoDO_" + start_date[:10] + "_" + str(date_DO[0][:10]) + ".csv"
df_DO.to_csv("Output/" + csv_name, sep=",", header=True, index=False)